## Build Larger Training Set

Now that this project looks somewhat promising. Let's build a proper dataset. The Steam API makes this extremely painful.

I don't ever want to have to do this again in my life, so we should try to pull down everything we could ever want and then just use that in the future.

In [1]:
import os
import json
import time
import datetime
import requests
from os.path import exists

In [1]:
import tqdm

In [ ]:
for tqdm.tqdm_notebook()

In [2]:
if 'STEAM_API_KEY' not in os.environ:
  print("No API Key :(")
else:
  print("Found API Key.")
  STEAM_API_KEY = os.environ['STEAM_API_KEY']

Found API Key.


In [3]:
base_url = 'https://api.steampowered.com'

# The maximum number of matches the API will return at once.
num_matches_per_request = 100

def __request(method, path, **kwargs):
  url = base_url + path
  kwargs.setdefault('params', dict()).update(key=STEAM_API_KEY)

  max_tries = 5
  num_tries = 0
  while num_tries < max_tries:
    try:
      response = requests.request(method, url, **kwargs)
      if response.status_code == 429:
        print("429. Backing off.")
        print(response.headers)
        time.sleep(60)
        num_tries += 1
        continue
      response_json = response.json()
      return response_json
    except:
      print("Error on request.")
    
    num_tries += 1
    time.sleep(60)


def get_match_history_by_seq_num(seq_num, num_matches, **params):
  path = '/IDOTA2Match_570/GetMatchHistoryBySequenceNum/V001'
  params.update(start_at_match_seq_num=seq_num)
  params.update(matches_requested=num_matches)
  return __request('get', path, params=params)

def get_most_recent_matches():
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  return __request('get', path)

def get_most_recent_seq_number():
  response = get_most_recent_matches()
  most_recent_matches = response['result']
  
  max_seq_num = -1
  for match in most_recent_matches:
    seq_num = match['match_seq_num']
    max_seq_num = max(match_seq_num, max_seq_num)
    
  return max_seq_num


In [4]:
# Remove unnecessary fields from game result. 
# We do this to minimize the space needed to store everything.
def delete_if_present(json, key):
  if key in json:
    del json[key]
    
def clean_match(match):
  delete_if_present(match, 'tower_status_radiant')
  delete_if_present(match, 'tower_status_dire')
  delete_if_present(match, 'barracks_status_radiant')
  delete_if_present(match, 'barracks_status_dire')
  delete_if_present(match, 'positive_votes')
  delete_if_present(match, 'negative_votes')
  delete_if_present(match, 'radiant_score')
  delete_if_present(match, 'dire_score')
  delete_if_present(match, 'picks_bans')
  for player in match['players']:
    delete_if_present(player, 'item_0')
    delete_if_present(player, 'item_1')
    delete_if_present(player, 'item_2')
    delete_if_present(player, 'item_3')
    delete_if_present(player, 'item_4')
    delete_if_present(player, 'item_5')
    delete_if_present(player, 'backpack_0')
    delete_if_present(player, 'backpack_1')
    delete_if_present(player, 'backpack_2')
    delete_if_present(player, 'item_neutral')
    delete_if_present(player, 'kills')
    delete_if_present(player, 'deaths')
    delete_if_present(player, 'assists')
    delete_if_present(player, 'last_hits')
    delete_if_present(player, 'denies')
    delete_if_present(player, 'gold_per_min')
    delete_if_present(player, 'level')
    delete_if_present(player, 'net_worth')
    delete_if_present(player, 'aghanims_scepter')
    delete_if_present(player, 'aghanims_shard')
    delete_if_present(player, 'moonshard')
    delete_if_present(player, 'xp_per_min')
    delete_if_present(player, 'additional_units')  

In [ ]:
bulk_download_path = "dota_2_bulk.tsv"

# We start at a sequence number from August 7, 2021 and work forward.
current_seq_num = 5125140861

if exists(bulk_download_path):
  print("Bulk file found. Scanning for last (largest) sequence number.")
  max_seq_num = current_seq_num
  with open(bulk_download_path, 'r') as bulk_file:
    for line in bulk_file:
        match_seq_num, _ = line.split('\t')
        max_seq_num = max(max_seq_num, int(match_seq_num))
        
  current_seq_num = max_seq_num

else:
  print("No Dota 2 Bulk File. Creating...")
  # Create the file, find the latest seq_num
  with open(bulk_download_path, 'w') as bulk_file:
    # Just create the file
    pass

print("Starting at: ", current_seq_num)
num_iters = 0
max_seq_num = current_seq_num

with open(bulk_download_path, 'a') as bulk_file:
  while True:
    if num_iters % 100 == 0:
      print("Iterations: ", num_iters, ". seq_num: ", current_seq_num)

    # Don't hit the API too frequently.
    time.sleep(2)
    response = get_match_history_by_seq_num(current_seq_num, num_matches_per_request)

    if response is None:
      print("Skipping. Response was None...")
      time.sleep(300)
      continue

    try:
      matches = response['result']['matches']
    except Exception as e:
      print("Bad error. This would normally break everything...")
      print(e)
      print(response)
      # Just wait 5 minutes before going again.
      time.sleep(300)
      continue

    if len(matches) != num_matches_per_request:
        print("Problem. Expected: " + num_matches_per_request + " matches. Actual: ", len(matches))
        print(response)
        break

    for match in matches:
        # Remove unneeded information to save space.
        clean_match(match)
        if match['human_players'] != 10:
            # We only want "real" games of Dota so we're ignoring
            # games without 10 human players.
            continue

        try:
          match_seq_num = match['match_seq_num']

          str_json = json.dumps(match)

          if "\t" in str_json:
            print("TAB DETECTED IN JSON!")
            print(str_json)
            continue

          output_str = str(match_seq_num) + "\t" + str_json + "\n"
          bulk_file.write(output_str)

          max_seq_num = max(max_seq_num, match_seq_num)
        except Exception as e:
          print(e)
          print("Error:", match)
          break


    current_seq_num = max_seq_num + 1
    num_iters += 1


print("Complete.")

Bulk file found. Scanning for last (largest) sequence number.
Starting at:  5187350273
Iterations:  0 . seq_num:  5187350273
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Skipping. Response was None...
Iterations:  0 . seq_num:  5187350273
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Skipping. Response was None...
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Skipping. Response was None...
Iterations:  100 . seq_num:  5187365888
Iterations:  200 . seq_num:  5187381765
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 06:17:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 06:17:49 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Con

Iterations:  3900 . seq_num:  5187961875
Error on request.
Iterations:  4000 . seq_num:  5187976473
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 09:24:41 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 09:24:41 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 09:27:45 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 09:27:45 GMT', 'Connection': 'keep-alive'}
Iterations:  4100 . seq_num:  5187991059
Iterations:  4200 . seq_num:  5188005443
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 09:35:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 09:35:50 GMT', 'Connection': 'keep-alive'}

Error on request.
Error on request.
Iterations:  8400 . seq_num:  5188633787
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 16:43:47 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 16:43:47 GMT', 'Connection': 'keep-alive'}
Iterations:  8500 . seq_num:  5188649995
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 16:49:52 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 16:49:52 GMT', 'Connection': 'keep-alive'}
Iterations:  8600 . seq_num:  5188666314
Iterations:  8700 . seq_num:  5188682357
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 16:57:47 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 16:57:47 GMT', 'Connecti

Iterations:  12600 . seq_num:  5189265771
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 20:42:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 20:42:51 GMT', 'Connection': 'keep-alive'}
Iterations:  12700 . seq_num:  5189279259
Iterations:  12800 . seq_num:  5189292750
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 20:50:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 20:50:46 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Wed, 22 Jun 2022 20:52:47 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 22 Jun 2022 20:52:47 GMT', 'Connection': 'keep-alive'}
Iterations:  12900 . seq_num:  5189305935
429. Backing 

Iterations:  16400 . seq_num:  5189836402
Iterations:  16500 . seq_num:  5189851907
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Thu, 23 Jun 2022 00:16:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Thu, 23 Jun 2022 00:16:49 GMT', 'Connection': 'keep-alive'}
Error on request.
Iterations:  16600 . seq_num:  5189867302
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Thu, 23 Jun 2022 00:37:52 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Thu, 23 Jun 2022 00:37:52 GMT', 'Connection': 'keep-alive'}
Iterations:  16700 . seq_num:  5189882820
Error on request.
Iterations:  16800 . seq_num:  5189898447
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Thu, 23 Jun 2022 01:02:44 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Dat

...

Iterations:  37400 . seq_num:  5192932730
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 05:38:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 05:38:51 GMT', 'Connection': 'keep-alive'}
Iterations:  37500 . seq_num:  5192947777
Iterations:  37600 . seq_num:  5192963095
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 05:44:52 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 05:44:52 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 05:47:45 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 05:47:45 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=

Error on request.
Iterations:  40700 . seq_num:  5193452982
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 08:40:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 08:40:49 GMT', 'Connection': 'keep-alive'}
Iterations:  40800 . seq_num:  5193468241
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 08:45:41 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 08:45:41 GMT', 'Connection': 'keep-alive'}
Iterations:  40900 . seq_num:  5193483537
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 08:50:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 08:50:46 GMT', 'Connection': 'keep-alive'}
Iterations:  41000 . seq_num:  519349

Iterations:  44200 . seq_num:  5193953262
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 14:42:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 14:42:50 GMT', 'Connection': 'keep-alive'}
Iterations:  44300 . seq_num:  5193966669
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 14:47:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 14:47:46 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 14:49:38 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 14:49:38 GMT', 'Connection': 'keep-alive'}
Iterations:  44400 . seq_num:  5193980195
429. Backing off.
{'Content-Type': 'text/html; charset=

Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Iterations:  48500 . seq_num:  5194619189
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 18:15:48 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 18:15:48 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 18:18:47 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 18:18:47 GMT', 'Connection': 'keep-alive'}
Iterations:  48600 . seq_num:  5194634094
Iterations:  48700 . seq_num:  5194649302
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 18:29:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 18:29:46

Error on request.
Error on request.
Error on request.
Error on request.
Skipping. Response was None...
Iterations:  51700 . seq_num:  5195071879
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 22:18:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 22:18:51 GMT', 'Connection': 'keep-alive'}
Iterations:  51800 . seq_num:  5195085766
Iterations:  51900 . seq_num:  5195099951
Iterations:  52000 . seq_num:  5195114730
Iterations:  52100 . seq_num:  5195129575
Iterations:  52200 . seq_num:  5195144889
Iterations:  52300 . seq_num:  5195160489
Iterations:  52400 . seq_num:  5195175948
Iterations:  52500 . seq_num:  5195191647
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Fri, 24 Jun 2022 22:52:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Fri, 24 Jun 2022 22:52:46 GMT', '

#### Not sure what that last error was. We should update the script to handle that in the future. For now I have over 5 million games of Dota downloaded so I'm going to leave it.

#### Problem at: 1264575 perhaps. We'll have to look into it.